# Assignment 3

In [1]:
import pandas as pd
import numpy as np

# Load MovieLens 100K dataset into a dataframe of pandas
names = ['user_id', 'item_id', 'rating', 'timestamp']
df = pd.read_csv('ml-100k/u.data', sep='\t', names=names)
df.head()

,user_id,item_id,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


In [2]:
# Select 500 most active users and 500 most active items from the dataset
n_most_active_users = 500
n_most_active_items = 500

user_ids = df.groupby('user_id').count().sort_values(by='rating', ascending=False).head(n_most_active_users).index
item_ids = df.groupby('item_id').count().sort_values(by='rating', ascending=False).head(n_most_active_items).index
df = df[(df['user_id'].isin(user_ids)) & (df['item_id'].isin(item_ids))]

In [3]:
# Map new internal ID for items
i_ids = df['item_id'].unique().tolist()
item_dict = dict(zip(i_ids, [i for i in range(len(i_ids))]))
df['item_id'] = df['item_id'].map(item_dict)

# Split Dataset

In [4]:
# The number of training users and active users
n_training_users = 300
n_active_users = n_most_active_users - n_training_users

# The number of GIVEN ratings for active users
GIVEN = 20

# Randomly select users from the most active users as training set
random_uids = np.random.choice(df.user_id.unique(), n_training_users, replace=False)
train_df = df[df['user_id'].isin(random_uids)]
# Map new internal ID for all users in the training set
u_ids = train_df['user_id'].unique().tolist()
user_dict = dict(zip(u_ids, [i for i in range(len(u_ids))]))
train_df['user_id'] = train_df['user_id'].map(user_dict)

# The rest of users are active users for testing
remain_df = df[~df['user_id'].isin(random_uids)]
# Map new internal ID for all active users
u_ids = remain_df['user_id'].unique().tolist()
user_dict = dict(zip(u_ids, [i for i in range(len(u_ids))]))
remain_df['user_id'] = remain_df['user_id'].map(user_dict)

# Randomly select GIVEN ratings for active users
active_df = remain_df.groupby('user_id').sample(n=GIVEN, random_state=1024)

test_df = remain_df[~remain_df.index.isin(active_df.index)]

<ipython-input-4-fb140fbf3b12>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df['user_id'] = train_df['user_id'].map(user_dict)
<ipython-input-4-fb140fbf3b12>:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  remain_df['user_id'] = remain_df['user_id'].map(user_dict)


In [5]:
# Convert the format of datasets to matrices
df_zeros = pd.DataFrame({'user_id': np.tile(np.arange(0, n_training_users), n_most_active_items), 'item_id': np.repeat(np.arange(0, n_most_active_items), n_training_users), 'rating': 0})
train_ds = df_zeros.merge(train_df, how='left', on=['user_id', 'item_id']).fillna(0.).pivot_table(values='rating_y', index='user_id', columns='item_id')

df_zeros = pd.DataFrame({'user_id': np.tile(np.arange(0, n_active_users), n_most_active_items), 'item_id': np.repeat(np.arange(0, n_most_active_items), n_active_users), 'rating': 0})
active_ds = df_zeros.merge(active_df, how='left', on=['user_id', 'item_id']).fillna(0.).pivot_table(values='rating_y', index='user_id', columns='item_id')
test_ds = df_zeros.merge(test_df, how='left', on=['user_id', 'item_id']).fillna(0.).pivot_table(values='rating_y', index='user_id', columns='item_id')

train_ds, active_ds, test_ds

(item_id  0    1    2    3    4    5    6    7    8    9    ...  490  491  492  \
 user_id                                                    ...                  
 0        0.0  0.0  0.0  0.0  5.0  0.0  0.0  0.0  0.0  4.0  ...  0.0  0.0  0.0   
 1        4.0  0.0  5.0  0.0  0.0  3.0  4.0  2.0  0.0  2.0  ...  0.0  2.0  0.0   
 2        0.0  4.0  0.0  0.0  0.0  0.0  3.0  0.0  0.0  4.0  ...  0.0  0.0  3.0   
 3        0.0  0.0  0.0  0.0  0.0  5.0  4.0  4.0  0.0  5.0  ...  3.0  0.0  0.0   
 4        0.0  0.0  0.0  3.0  0.0  0.0  0.0  4.0  0.0  0.0  ...  4.0  3.0  0.0   
 ...      ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...   
 295      4.0  0.0  3.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
 296      0.0  0.0  0.0  5.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
 297      0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  3.0  4.0  0.0   
 298      0.0  0.0  0.0  3.0  0.0  0.0  0.0  4.0  0.0  0.0  ...  0.0  0.0  5.0   
 299      0.0  0

In [6]:
# Predicting All Missing Data in training set
imputed_train_ds = train_ds.values.copy()

# Your implementation to predict the missing values
(Put all your implementation for your algorithm in the following cell only to handle the missing values; )

In [7]:
## Put all your implementation for your solutioin in this cell only to predict the missing values; 
## NOTE 1: DO NOT change anything in the rest of the cells in this framework, 
## otherwise the changes might cause errors and make your implementation invalid.

## Note 2: 
## The user-item rating matrix is imputed_train_ds, 
## and the missing values are those 0s in imputed_train_ds. 
## You are required to predict them by using the solution in the given report. 

## The following parameters are required in the given report, 
## which is named "Effective Missing Data Prediction for Collaborative Filtering", 
## and you will need to use them. But, please do not change their values. 
LAMBDA = 0.7    # λ
GAMMA = 10      # γ
DELTA = 10      # δ
ITA = 0.7       # η
THETA = 0.7     # θ
EPSILON = 1e-9

train_user_pearson_corr = np.zeros((imputed_train_ds.shape[0], imputed_train_ds.shape[0]))

# compute Pearson Correlation Coefficient of all pairs of users in the training set
# loop through the rows of every pair of users
for i, user_i in enumerate(imputed_train_ds):
    for j, user_j in enumerate(imputed_train_ds):

        # ratings given by two users respectively
        mask_i = user_i > 0
        mask_j = user_j > 0

        # indices of items co-rated by both users
        corrated_index = np.intersect1d(np.where(mask_i), np.where(mask_j))
        
        # skip the current loop if there is no co-rated items
        if len(corrated_index) == 0:
            continue

        # average rating given by two users
        mean_user_i = np.sum(user_i) / (np.sum(np.clip(user_i, 0, 1)) + EPSILON)
        mean_user_j = np.sum(user_j) / (np.sum(np.clip(user_j, 0, 1)) + EPSILON)

        # normalise ratings
        norm_i = user_i[corrated_index] - mean_user_i
        norm_j = user_j[corrated_index] - mean_user_j

        # compute pearson corr coeff
        r_ui_sub_r_i_sq = np.square(norm_i)
        r_uj_sub_r_j_sq = np.square(norm_j)

        r_ui_sum_sqrt = np.sqrt(np.sum(r_ui_sub_r_i_sq))
        r_uj_sum_sqrt = np.sqrt(np.sum(r_uj_sub_r_j_sq))

        sim = np.sum(norm_i * norm_j) / (r_ui_sum_sqrt * r_uj_sum_sqrt + EPSILON)

        # significance weighting
        weighted_sim = (min(len(corrated_index), GAMMA) / GAMMA) * sim

        train_user_pearson_corr[i][j] = weighted_sim

train_item_pearson_corr = np.zeros((imputed_train_ds.shape[1], imputed_train_ds.shape[1]))

# compute Pearson Correlation Coefficient of all pairs of items in the training set
# transpose the user-item matrix into item-user matrix
# loop through the rows of every pair of items
for i, item_i in enumerate(imputed_train_ds.T):
    for j, item_j in enumerate(imputed_train_ds.T):

        # ratings received by two items respectively
        mask_i = item_i > 0
        mask_j = item_j > 0

        # indices of users that rated both items
        corrated_index = np.intersect1d(np.where(mask_i), np.where(mask_j))
        
        # skip the current loop if there is no co-rated users
        if len(corrated_index) == 0:
            continue

        # average rating received by two items
        mean_item_i = np.sum(item_i) / (np.sum(np.clip(item_i, 0, 1)) + EPSILON)
        mean_item_j = np.sum(item_j) / (np.sum(np.clip(item_j, 0, 1)) + EPSILON)

        # normalise ratings
        norm_i = item_i[corrated_index] - mean_item_i
        norm_j = item_j[corrated_index] - mean_item_j

        # compute pearson corr coeff
        r_ui_sub_ri_sq = np.square(norm_i)
        r_uj_sub_rj_sq = np.square(norm_j)

        r_ui_sum_sqrt = np.sqrt(np.sum(r_ui_sub_ri_sq))
        r_uj_sum_sqrt = np.sqrt(np.sum(r_uj_sub_rj_sq))

        sim = np.sum(norm_i * norm_j) / (r_ui_sum_sqrt * r_uj_sum_sqrt + EPSILON)

        # significance weighting
        weighted_sim = (min(len(corrated_index), DELTA) / DELTA) * sim

        train_item_pearson_corr[i][j] = weighted_sim
        
# predict missing values
for (i, j), rating in np.ndenumerate(imputed_train_ds):
    if rating == 0:
        # user-based prediction
        # find the user neighbours that pass the threshold, exclude the current user i itself
        sim_user_ids = np.argsort(train_user_pearson_corr[i])
        sim_user_ids = sim_user_ids[0:len(sim_user_ids) - 1]
        sim_user_ids = sim_user_ids[train_user_pearson_corr[i][sim_user_ids] > ITA]
        
        # the similarity values of user neighbours
        sim_val = train_user_pearson_corr[i][sim_user_ids]
        
        # the average rating given by current user i
        user_mean = np.sum(train_ds.values[i]) / (np.sum(np.clip(train_ds.values[i], 0, 1)) + EPSILON)
        
        # the average rating given by each user neighbour
        sim_users = train_ds.values[sim_user_ids]
        sim_user_mean = np.sum(sim_users, axis=1) / (np.sum(np.clip(sim_users, 0, 1), axis=1) + EPSILON)
        
        # select the user neighbours who rated item j
        mask_rated_j = sim_users[:, j] > 0
        
        # for each user neighbour who rated item j
        # similarity of this neighbour * (this neighbour's rating on item j - this neighbour's average rating)
        sim_r_sum_mean = sim_val[mask_rated_j] * (sim_users[mask_rated_j, j] - sim_user_mean[mask_rated_j])
        
        # compute user-based prediction
        user_pred = user_mean + np.sum(sim_r_sum_mean) / (np.sum(sim_val[mask_rated_j]) + EPSILON)
        
        # item-based prediction
        # find the item neighbours that pass the threshold, exclude the current item j itself
        sim_item_ids = np.argsort(train_item_pearson_corr[j])
        sim_item_ids = sim_item_ids[0:len(sim_item_ids) - 1]
        sim_item_ids = sim_item_ids[train_item_pearson_corr[j][sim_item_ids] > THETA]
        
        # the similarity values of item neighbours
        sim_item_val = train_item_pearson_corr[j][sim_item_ids]
        
        # the average rating received by current item j
        item_mean = np.sum(train_ds.T.values[j]) / (np.sum(np.clip(train_ds.T.values[j], 0, 1)) + EPSILON)
        
        # the average rating received by each item neighbour
        sim_items = train_ds.T.values[sim_item_ids]
        sim_item_mean = np.sum(sim_items, axis=1) / (np.sum(np.clip(sim_items, 0, 1), axis=1) + EPSILON)
        
        # select the item neighbours which are rated by user i
        mask_ratedby_i = sim_items[:, i] > 0
        
        # for each item neighbour which is rated by user i
        # similarity of this neighbour * (this neighbour's rating from user i - this neighbour's average rating)
        sim_item_r_sum_mean = sim_item_val[mask_ratedby_i] * (sim_items[mask_ratedby_i, i] - sim_item_mean[mask_ratedby_i])
        
        # compute item-based prediction
        item_pred = item_mean + np.sum(sim_item_r_sum_mean) / (np.sum(sim_item_val[mask_ratedby_i]) + EPSILON)
        
        # compute final prediction
        if len(sim_user_ids) > 0 and len(sim_item_ids) > 0:
            rating_pred = LAMBDA * user_pred + (1 - LAMBDA) * item_pred
        elif len(sim_user_ids) > 0 and len(sim_item_ids) == 0:
            rating_pred = user_pred
        elif len(sim_user_ids) == 0 and len(sim_item_ids) > 0:
            rating_pred = item_pred
        else:
            rating_pred = 0
        
        # enforce the valid range of ratings
        rating_pred = np.clip(rating_pred, 0, 5)
        
        imputed_train_ds[i][j] = rating_pred

# Evaluation

### Compute Pearson Correlation Coefficient of All Pairs of Items between active set and imputed training set

In [8]:
imputed_train_ds = pd.DataFrame(imputed_train_ds)
imputed_train_ds

,0,1,2,3,4,5,6,7,8,9,...,490,491,492,493,494,495,496,497,498,499
0,4.156250,3.441860,0.000000,3.845528,5.000000,3.367816,3.811594,3.280202,4.029439,4.000000,...,3.857447,2.342491,0.000000,3.892857,2.893617,2.351351,1.604487,4.607317,2.677419,4.322316
1,4.000000,4.166656,5.000000,3.845528,0.653680,3.000000,4.000000,2.000000,2.029439,2.000000,...,2.032447,2.000000,0.000000,4.185540,2.893617,2.351351,2.297297,4.000000,2.677419,1.817426
2,3.556735,4.000000,3.241596,3.361906,3.897537,3.573816,3.000000,3.547532,3.136154,4.000000,...,3.102812,2.910995,3.000000,2.810390,2.976333,2.913653,2.999050,2.954496,3.265319,3.168247
3,4.156250,3.794716,0.000000,3.845528,4.539393,5.000000,4.000000,4.000000,2.745197,5.000000,...,3.000000,2.966985,0.000000,3.892857,5.000000,2.351351,0.604487,3.871285,2.677419,4.582353
4,3.938321,3.436283,0.000000,3.000000,3.362238,4.117816,4.006083,4.000000,2.819823,0.000000,...,4.000000,3.000000,0.000000,3.000000,1.560284,2.351351,2.124588,3.739458,2.677419,2.582353
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,4.000000,3.369932,3.000000,3.635477,3.676492,3.492163,3.534387,3.465879,3.409725,3.545455,...,2.947877,3.186920,3.545455,3.649675,3.349903,2.889075,3.263164,3.492484,3.285044,3.398961
296,4.276726,4.062409,4.328358,5.000000,4.356111,4.040196,4.173329,4.170834,4.238683,4.328358,...,3.917085,4.334953,4.328358,4.327253,3.585436,3.735256,3.719040,4.142046,3.833077,3.792057
297,4.156250,1.000000,0.000000,3.845528,2.778642,3.367816,3.042363,3.803279,3.452508,0.000000,...,3.000000,4.000000,0.000000,3.892857,3.560284,2.351351,2.297297,3.707317,2.677419,3.817426
298,4.204328,2.932524,2.944444,3.000000,3.118254,2.996456,3.204589,4.000000,2.989018,3.224932,...,1.268280,2.850397,5.000000,3.228968,2.929196,3.000000,2.750300,3.457090,2.864337,2.835817


In [9]:
active_user_pearson_corr = np.zeros((active_ds.shape[0], train_ds.shape[0]))

# Compute Pearson Correlation Coefficient of All Pairs of Users between active set and imputed training set
for i, user_i_vec in enumerate(active_ds.values):
    for j, user_j_vec in enumerate(imputed_train_ds.values):
        
        # ratings corated by the current pair of users
        mask_i = user_i_vec > 0
        mask_j = user_j_vec > 0

        # corrated item index, skip if there are no corrated ratings
        corrated_index = np.intersect1d(np.where(mask_i), np.where(mask_j))
        if len(corrated_index) == 0:
            continue

        # average value of user_i_vec and user_j_vec
        mean_user_i = np.sum(user_i_vec) / (np.sum(np.clip(user_i_vec, 0, 1)) + EPSILON)
        mean_user_j = np.sum(user_j_vec) / (np.sum(np.clip(user_j_vec, 0, 1)) + EPSILON)

        # compute pearson corr
        user_i_sub_mean = user_i_vec[corrated_index] - mean_user_i
        user_j_sub_mean = user_j_vec[corrated_index] - mean_user_j

        r_ui_sub_r_i_sq = np.square(user_i_sub_mean)
        r_uj_sub_r_j_sq = np.square(user_j_sub_mean)

        r_ui_sum_sqrt = np.sqrt(np.sum(r_ui_sub_r_i_sq))
        r_uj_sum_sqrt = np.sqrt(np.sum(r_uj_sub_r_j_sq))

        sim = np.sum(user_i_sub_mean * user_j_sub_mean) / (r_ui_sum_sqrt * r_uj_sum_sqrt + EPSILON)

        # significance weighting
        weighted_sim = (min(len(corrated_index), GAMMA) / GAMMA) * sim

        active_user_pearson_corr[i][j] = weighted_sim

active_user_pearson_corr

array([[ 0.13490673,  0.16490938,  0.30615924, ..., -0.0122867 ,
         0.36650816, -0.19250559],
       [ 0.19622043,  0.37890189, -0.04285867, ...,  0.47183079,
         0.39774746,  0.24787326],
       [ 0.21572646, -0.08744522, -0.0824993 , ...,  0.35087124,
        -0.18021959,  0.12692653],
       ...,
       [ 0.43885845,  0.18940392,  0.15292347, ...,  0.30316112,
        -0.32521344,  0.4774363 ],
       [ 0.42289589,  0.42497621,  0.23411608, ...,  0.18484718,
         0.30470159,  0.0765803 ],
       [ 0.38799218,  0.04256656,  0.07215277, ..., -0.25869053,
        -0.064573  ,  0.2929065 ]])

## Predict Ratings of Testing Set

In [10]:
K = 10

test_ds_pred = np.zeros_like(test_ds.values)

for (i, j), rating in np.ndenumerate(test_ds.values):

    if rating > 0:

        sim_user_ids = np.argsort(active_user_pearson_corr[i])[-1:-(K + 1):-1]

        #==================user-based==================#
        # the coefficient values of similar users
        sim_val = active_user_pearson_corr[i][sim_user_ids]

        # the average value of the current user's ratings
        sim_users = imputed_train_ds.values[sim_user_ids]
        user_mean = np.sum(active_ds.values[i]) / (np.sum(np.clip(active_ds.values[i], 0, 1)) + EPSILON)
        sim_user_mean = np.sum(sim_users, axis=1) / (np.sum(np.clip(sim_users, 0, 1), axis=1) + EPSILON)

        # select the users who rated item j
        mask_rated_j = sim_users[:, j] > 0
        
        # sim(u, v) * (r_vj - mean_v)
        sim_r_sum_mean = sim_val[mask_rated_j] * (sim_users[mask_rated_j, j] - sim_user_mean[mask_rated_j])
        
        user_based_pred = user_mean + np.sum(sim_r_sum_mean) / (np.sum(sim_val[mask_rated_j]) + EPSILON)
        user_based_pred = np.clip(user_based_pred, 0, 5)

        test_ds_pred[i][j] = user_based_pred
        
test_ds_pred


array([[0.        , 0.        , 0.        , ..., 0.        , 2.77067163,
        0.        ],
       [0.        , 3.3410317 , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 4.65854001, ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 3.5398246 , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 4.46200736, ..., 0.        , 0.        ,
        0.        ]])

## Compute MAE and RMSE

In [11]:
# MAE
MAE = np.sum(np.abs(test_ds_pred - test_ds.values)) / np.sum(np.clip(test_ds.values, 0, 1))

# RMSE
RMSE = np.sqrt(np.sum(np.square(test_ds_pred - test_ds.values)) / np.sum(np.clip(test_ds.values, 0, 1)))

print("MAE: {}, RMSE: {}" .format(MAE, RMSE))

MAE: 0.7288451770359519, RMSE: 0.9353799951033979
